<a href="https://colab.research.google.com/github/PrasadK97/Deep_Learning_Assignment_3/blob/main/seq2seq_model_without_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import csv
import requests

In [ ]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=b4ed3f2783806a40eb547ea18a85aaa1efbdbf77a22bfb8aec04031f050a4e67
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb
from io import open
import unicodedata
import pandas as pd
import numpy as np
import string
import re
import random
import time

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Function to read and split the CSV file
def read_csv_file(url):
    response = requests.get(url)
    csv_data = response.text
    lines = csv_data.strip().split('\n')
    pairs = [line.split(',') for line in lines]
    return pairs

In [ ]:


class Language:
    def __init__(self, name):
        self.name = name
        self.letter2index = {}
        self.letter2count = {}
        self.index2letter = {0: "SOS", 1: "EOS"}
        self.n_letters = 2  # Count SOS and EOS

    def letter(self, letter): # making a dictionary of letters and their counts
        if letter not in self.letter2index:
            self.letter2index[letter] = self.n_letters
            self.letter2count[letter] = 1
            self.index2letter[self.n_letters] = letter
            self.n_letters += 1
        else:
            self.letter2count[letter] += 1

    def word(self, letter): # adding a word to the dictionary
        for letter in letter:
            self.letter(letter)

    def decode(self, target):
        return ' '.join([self.index2letter[i.get] for i in target])


In [ ]:
def read(language1, language2, reverse=False): # read the file and make a dictionary of words of both languageuages

    # Read the file and split into lines
    # URLs of the CSV files on GitHub
    train_url = 'https://github.com/PrasadK97/Deep_Learning_Assignment_3/raw/main/aksharantar_sampled/mar/mar_train.csv'
    val_url = 'https://github.com/PrasadK97/Deep_Learning_Assignment_3/raw/main/aksharantar_sampled/mar/mar_valid.csv'

    # Read and split the train CSV file
    train_pairs = read_csv_file(train_url)

    # Read and split the validation CSV file
    val_pairs = read_csv_file(val_url)
    
    input_language = Language(language1)
    output_language = Language(language2)

    for pair in train_pairs:
        input_language.word(pair[0])
        output_language.word(pair[1])
    
    for pair in val_pairs:
        input_language.word(pair[0])
        output_language.word(pair[1])

    return train_pairs, val_pairs, input_language, output_language

In [ ]:
sos = 0
eos = 1
def indices(language, word):
    return [language.letter2index[letter] for letter in word]


def tensor_word(language, word): 
    indexes = indices(language, word)
    indexes.append(eos)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensors_Pair(pair, input_language, output_language): 
    input_tensor = tensor_word(input_language, pair[0])
    target_tensor = tensor_word(output_language, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
# ENCODER MODEL

class Encoder(nn.Module):
    def __init__(self, type, input_size, emb_size, hidden_size, p, num_layers):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, emb_size)
        self.rnn = nn.RNN(emb_size, hidden_size, num_layers, dropout = p)
        self.gru = nn.GRU(emb_size, hidden_size, num_layers, dropout = p)
        self.lstm = nn.LSTM(emb_size, hidden_size, num_layers, dropout = p)
        self.type_t = type

    def forward(self, input, hidden):
        embedded = self.dropout(self.embedding(input)).view(1, 1, -1)
        output = embedded
        
      
        if self.type_t == 'RNN':
            output, hidden = self.rnn(output, hidden)
        elif self.type_t == 'GRU':
            output, hidden = self.gru(output, hidden)
        elif self.type_t == 'LSTM':
            output, hidden = self.lstm(output, hidden)

        return output, hidden
    
    def initHidden(self): 
        if self.type_t == 'LSTM':
            return (torch.zeros(self.num_layers, 1, self.hidden_size, device=device), torch.zeros(self.num_layers, 1, self.hidden_size, device=device))
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)


# DECODER MODEL

class Decoder(nn.Module): 
    def __init__(self, type, output_size, embedding_size, hidden_size, p, num_layers):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout = p)
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers, dropout = p)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.type_t = type

    def forward(self, input, hidden):
        output = self.dropout(self.embedding(input)).view(1, 1, -1) 
        output = F.relu(output) 
        
       
        if self.type_t == 'RNN':
            output, hidden = self.rnn(output, hidden)
        elif self.type_t == 'GRU':
            output, hidden = self.gru(output, hidden)
        elif self.type_t == 'LSTM':
            output, hidden = self.lstm(output, hidden)

        
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def inithidden(self): 
        if self.type_t == 'LSTM':
            return (torch.zeros(self.num_layers, 1, self.hidden_size, device=device), torch.zeros(self.num_layers, 1, self.hidden_size, device=device))
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

In [ ]:
# Training the model

class Train(): 
    def __init__(self, train_data, encoder, decoder, criterion, tfr = 0.5):
        self.train_data = train_data
        self.encoder = encoder
        self.decoder = decoder
        self.criterion = criterion
        self.tfr = tfr
        self.train_pairs, self.val_pairs, self.input_lang, self.output_lang = readLang('eng', 'hin')
        self.training_pairs = [tensorsFromPair(self.train_pairs[i], self.input_lang, self.output_lang) for i in range(len(self.train_pairs))]

    def train(self, input_tensor, target_tensor, encoder_optimizer, decoder_optimizer):
        encoder_hidden = self.encoder.inithidden()
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        encoder_outputs = torch.zeros(50, self.encoder.hidden_size, device=device)

        loss = 0

        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)

        for i in range(input_length): 
            encoder_output, encoder_hidden = self.encoder(input_tensor[i], encoder_hidden)
            encoder_outputs[i] += encoder_output[0, 0]

        decoder_input = torch.tensor([[sos]], device=device)
        decoder_hidden = encoder_hidden 

        teacher_forc = True if random.random() < self.tfr else False

        if teacher_forc: 
            for i in range(target_length):
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                loss += self.criterion(decoder_output, target_tensor[i])
                decoder_input = target_tensor[i] 

        else:
            for i in range(target_length):
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1) 
                decoder_input = topi.squeeze().detach() 
                loss += self.criterion(decoder_output, target_tensor[i])
                if decoder_input.item() == eos: 
                    break

        loss.backward() 

        encoder_optimizer.step()
        decoder_optimizer.step()

        return loss.item() / target_length


    def trainIters(self, optimizer, learning_rate, n_iters = 50, print_every = 50, epochs=-1):
        start = time.time()
        print_loss_total = 0

        if optimizer == 'sgd':

            encoder_optimizer = optim.sgd(self.encoder.parameters(), lr = learning_rate)
            decoder_optimizer = optim.sgd(self.decoder.parameters(), lr = learning_rate)


        elif optimizer == 'RMSprop':

            encoder_optimizer = optim.RMSprop(self.encoder.parameters(), lr = learning_rate)
            decoder_optimizer = optim.RMSprop(self.decoder.parameters(), lr = learning_rate)
            
        elif optimizer == 'adam':

            encoder_optimizer = optim.adam(self.encoder.parameters(), lr = learning_rate)
            decoder_optimizer = optim.adam(self.decoder.parameters(), lr = learning_rate)

       

        elif optimizer == 'Nadam':

            encoder_optimizer = optim.Nadam(self.encoder.parameters(), lr = learning_rate)
            decoder_optimizer = optim.Nadam(self.decoder.parameters(), lr = learning_rate)

        if epochs != -1:

            n_iters = len(self.train_pairs)
        else:

            train_loss_total = 0
            for iter in tqdm(range(1, n_iters+1)):
                training_pair = self.training_pairs[iter - 1]
                input_tensor = training_pair[0]
                target_tensor = training_pair[1]
                loss = self.train(input_tensor, target_tensor, encoder_optimizer, decoder_optimizer)
                train_loss_total += loss

                if iter % print_every == 0:
                    print_loss_avg = print_loss_total / print_every
                    print_loss_total = 0
                    print('%s (%d %d%%) %.4f' % (time_since(start, iter/n_iters), iter, iter/n_iters*100, print_loss_avg))
            train_acc = self.evaluate_data(self.train_pairs)
            valid_acc = self.evaluate_data(self.val_pairs)
            return train_acc, valid_acc

        loss_train = []
        val_acc = []
        acc_train = []
        for j in range(epochs):
            train_loss_total = 0
            for iter in tqdm(range(1, n_iters+1)):
                training_pair = self.training_pairs[iter - 1]
                input_tensor = training_pair[0]
                target_tensor = training_pair[1]
                loss = self.train(input_tensor, target_tensor, encoder_optimizer, decoder_optimizer)
                train_loss_total += loss
                print_loss_total += loss

                if iter % print_every == 0:
                    print_loss_avg = print_loss_total / print_every
                    print_loss_total = 0
                    print('%s (%d %d%%) %.4f' % (timeSince(start, iter/n_iters), iter, iter/n_iters*100, print_loss_avg))
            train_acc = self.evaluate_data(self.train_pairs)
            valid_acc = self.evaluate_data(self.val_pairs)
            loss_train.append(train_loss_total / n_iters)
            val_acc.append(valid_acc)
            acc_train.append(train_acc)
            wandb.log({'train_loss': train_loss_total / n_iters, 'train_acc': train_acc, 'valid_acc': valid_acc})
        return loss_train, acc_train, val_acc
                    

    def evaluate(self, word):
        with torch.no_grad():
            input_tensor = tensor_word(self.input_lang, word)
            input_length = input_tensor.size()[0]
            encoder_hidden = self.encoder.initHidden()

            encoder_outputs = torch.zeros(50, self.encoder.hidden_size, device=device)

            for i in range(input_length): # encoding a word
                encoder_output, encoder_hidden = self.encoder(input_tensor[i], encoder_hidden)
                # print(encoder_output.shape)
                encoder_outputs[i] += encoder_output[0, 0]

            decoder_input = torch.tensor([[sos]], device=device)
            decoder_hidden = encoder_hidden # encoder shares its hidden layer with decoder

            decoded_word = ''

            for i in range(50):
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1) # top k predictions
                if topi.item() == eos:
                    break
                else:
                    decoded_word += (self.output_lang.index2letter[topi.item()])
                decoder_input = topi.squeeze().detach() # detach from history as input

            return decoded_word
        
    def evaluate_data(self, data):
        acc = 0
        for word,target in data:
            acc += (self.evaluate(word) == target)
        return acc / len(data)

train_pairs, val_pairs, input_lang, output_lang = readLang('eng', 'mar')

In [ ]:
import time
import math


def as_min(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_min(s), as_min(rs))


In [ ]:
sweep_config = {
    'method': 'random', 
    'metric': {
        'name': 'valid_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'optimizer': {
            'values': ['SGD', 'Adam', 'RMSprop', 'NAdam']
        },
        'learning_rate': {
            'values': [1e-4, 5e-4, 0.001, 0.005, 0.01]
        },
        'epochs': {
            'values': [5, 10, 15, 20]
        },
        'hid_layers': {
            'values': [1, 2, 3, 4]
        },
        'emb_size': {
            'values': [64, 128, 256, 512]
        },
        'hidden_size': {
            'values': [64, 128, 256, 512]
        },
        'dropout': {
            'values': [0, 0.1, 0.2, 0.3, 0.4]
        },
        'type_t': {
            'values': ['RNN', 'LSTM', 'GRU']
        }
    }
}


In [ ]:
def run():
  config_defaults = {
        'optimizer': 'Adam',
        'learning_rate': 0.005,
        'epochs': 10,
        'hid_layers': 1,
        'emb_size': 256,
        'hidden_size': 256,
        'dropout': 0.1,
        'type_t': 'GRU'
  }
  wandb.init(config=config_defaults)
  config = wandb.config
  encoder = Encoder(config.type_t, input_lang.n_letters, config.emb_size, config.hidden_size, config.dropout, config.hid_layers).to(device)
  decoder = Decoder(config.type_t, output_lang.n_letters, config.emb_size, config.hidden_size, config.dropout, config.hid_layers).to(device)
  train = Train(train_pairs, encoder, decoder, nn.NLLLoss(), config.dropout)
  train.trainIters(config.optimizer, config.learning_rate,print_every= 1000, epochs=config.epochs)

  wandb.finish()

sweep_id = wandb.sweep(sweep_config, project='Assignment_3')
wandb.agent(sweep_id, function=run, count=10)

Create sweep with ID: 5hfzg3eg
Sweep URL: https://wandb.ai/prasad_kokitkar/Assignment_3/sweeps/5hfzg3eg


wandb: Agent Starting Run: jyajzsvt with config:
wandb: 	dropout: 0
wandb: 	emb_size: 256
wandb: 	epochs: 10
wandb: 	hid_layers: 1
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.01
wandb: 	optimizer: NAdam
wandb: 	type_t: RNN
wandb: Currently logged in as: prasad_kokitkar. Use `wandb login --relogin` to force relogin


  2%|▏         | 1009/51200 [00:17<12:13, 68.40it/s]

0m 17s (- 14m 40s) (1000 1%) 3.7265


  4%|▍         | 2010/51200 [00:33<13:38, 60.06it/s]

...

 98%|█████████▊| 50008/51200 [13:33<00:17, 69.34it/s]

31m 2s (- 0m 44s) (50000 97%) 3.1989


100%|█████████▉| 51012/51200 [13:49<00:02, 66.68it/s]

31m 19s (- 0m 7s) (51000 99%) 3.3407


100%|██████████| 51200/51200 [13:52<00:00, 61.51it/s]
wandb: Ctrl + C detected. Stopping sweep.
